In [1]:
# IMPORT Thư viện
#  pip install numpy pandas jupyter notebook openpyxl xlrd
import pandas as pd
import os
import openpyxl
import xlrd
import numpy as np

In [2]:
# IMPORT CÁC FILE (5 FILES): TỔNG NHẬP XUẤT TỒN ; GIÁ BÁN; MUA HÀNG ; BÁN HÀNG; INPUT

# IMPORT 1 Tổng hợp nhập xuất tồn -> RENAME 
Data_Ton = pd.read_excel("Tổng hợp Nhập - Xuất - Tồn YTD.xls",usecols="A,F,K:L",skipfooter=1,header=5)
Data_Ton=Data_Ton.rename(columns={"Unnamed: 0":"ID",
                                "Unnamed: 5":"ID_Category",
                                "Số lượng.2":"Tổng xuất", 
                                "Giá trị.2":"Giá trị xuất" })

#Import INPUT Quotation
Data_Request=pd.read_excel("Input.xlsx",sheet_name='Data', usecols="B:I")


#Import 2: Báo cáo giá bán tồn kho
Data_GiaBanTonKho=pd.read_excel("Báo cáo giá bán, tồn kho.xls",usecols="C,F,J",header=4,skipfooter=1)
Data_GiaBanTonKho=Data_GiaBanTonKho.rename(columns={"Mã số":"ID"})
Data_GiaBanTonKho.dropna(how='all',inplace=True)


#Import 3: Sổ chi tiết bán hàng
Data_BanHang=pd.read_excel("Sổ chi tiết bán hàng.xls",usecols="F,M",skipfooter=1,header=6)
Data_BanHang=Data_BanHang.rename(columns={"Unnamed: 0":"Ngày bán",
                                        "Unnamed: 5":"ID" ,
                                        "Giá bán":"Giá bán giảm"})


#Import 4: Sổ chi tiết mua hàng
Data_MuaHang=pd.read_excel("Sổ chi tiết mua hàng (2).xls",usecols="A,C,F,I:J",header=5,skipfooter=1)
Data_MuaHang=Data_MuaHang.rename(columns={"Unnamed: 0":"Ngày nhập",
                                        "Unnamed: 2":"Nhà cung cấp",
                                        "Unnamed: 5":"ID",
                                        "Số lượng":"Tổng nhập",
                                        "Đơn giá":"Đơn giá nhập"})


In [3]:
# TẠO BỘ LỌC CHO SỔ MUA HÀNG: GIÁ NHẬP NHỎ NHẤT; NGÀY NHẬP GẦN NHẤT
# # FILTER: Đơn giá nhập >0; Tổng nhập > 0; 
# Data_MuaHang=Data_MuaHang.loc[(Data_MuaHang['Đơn giá nhập']>0) & (Data_MuaHang['Tổng nhập']>0)].sort_values('Đơn giá nhập')
# # Tạo giá nhập nhỏ nhất
# Min_value=Data_MuaHang.groupby('ID')['Đơn giá nhập'].min()
# Data_MuaHang=Data_MuaHang.merge(Min_value,on='ID',suffixes=('', '_min'))
# # -> Filter theo ID có đơn giá nhập nhỏ nhất
# Data_MuaHang = Data_MuaHang[Data_MuaHang['Đơn giá nhập']==Data_MuaHang['Đơn giá nhập_min']].drop('Đơn giá nhập_min', axis=1)

#-> Filter Mã hàng có ngày nhập gần nhất
Date_max=Data_MuaHang.groupby('ID')['Ngày nhập'].max()
Data_MuaHang=Data_MuaHang.merge(Date_max,on='ID',suffixes=('', '_max'))
# -> Filter theo ID có đơn giá nhập nhỏ nhất
Data_MuaHang = Data_MuaHang[Data_MuaHang['Ngày nhập']==Data_MuaHang['Ngày nhập_max']].drop('Ngày nhập_max', axis=1)


# Data_MuaHang.loc[Data_MuaHang['ID']=="90913-02105"]
# Data_MuaHang.loc[Data_MuaHang['ID']=="T24NC50WB"]

In [4]:
# TẠO BỘ LỌC CHO SỔ BÁN HÀNG: GIÁ GIẢM NHỎ NHẤT;
Data_BanHang = Data_BanHang[Data_BanHang['Giá bán giảm'] == Data_BanHang.groupby('ID')['Giá bán giảm'].transform('min')]
Data_BanHang = Data_BanHang.drop_duplicates(['ID','Giá bán giảm'], keep="first")


In [5]:
# TẠO BỘ LỌC CHO NHÀ SẢN XUẤT: GIÁ NHỎ NHẤT;

#-> Filter Mã hàng có ngày nhập gần nhất
UnitPrice_Min=Data_Request.groupby(['ID'])['Unit price'].min()
Data_Request=Data_Request.merge(UnitPrice_Min,on='ID',suffixes=('', '_min'))
# -> Filter theo ID có đơn giá nhập nhỏ nhất
Data_Request = Data_Request[Data_Request['Unit price']==Data_Request['Unit price_min']].drop('Unit price_min', axis=1)
# Data_Request_2.loc[Data_Request_2['ID']=='04152-YZZA4']


In [6]:
# TÍNH TOÁN GIÁ VỐN
# Calcuation on Tong Hop Ton Kho -------------------------------------------

# Gia Von= Gia Tri Xuat/SL Xuat * 1.05
Data_Ton['Giá vốn']=np.around(Data_Ton['Giá trị xuất']/Data_Ton['Tổng xuất']*1.05,0)


In [7]:
# TỔNG HỢP DỮ LIỆU

# Merge Request -> Gia Von
Data_Request= pd.merge(Data_Request,Data_Ton[['ID','Giá vốn','Tổng xuất']],on='ID', how='left')
# # Merge  (Báo cáo nhập xuất tồn)
Data_Request=pd.merge(Data_Request,Data_GiaBanTonKho[['ID','Giá bán','Tổng tồn']],how='left',on='ID')
# # # # Merge Giá bán hàng
Data_Request=pd.merge(Data_Request,Data_BanHang,how='inner',on='ID')
# # Merge Giá mua hàng
Data_Request=pd.merge(Data_Request,Data_MuaHang,how='left',on='ID')


In [8]:
# TÍNH TOÁN CÁC HỆ SỐ
# Calcuation on Request -------------------------------------------


#VND/US=24,000; he so = 1.13

Data_Request['Unit Price VND'] = np.around(Data_Request['Unit price']*24000*1.13,0)
Data_Request['Tỷ lệ Giá vốn chênh lệch'] = np.around((Data_Request['Unit Price VND']-Data_Request['Giá vốn'])/Data_Request['Giá vốn'],2)

In [9]:
# MẶC CẢ GIÁ VỚI NHÀ CUNG CẤP; RATE = 5%
RateDeal=0.05
CostTrans=0.13
RateVNDUSD=24000

# GIÁ MẶC CẢ VND
# Data_Request['Giá mặc cả VND']=np.around(Data_Request.loc[Data_Request['Tỷ lệ Giá vốn chênh lệch']>=0.0000]['Đơn giá nhập']*(1-RateDeal),0)
# Data_Request['Giá mặc cả VND']=np.around(Data_Request.loc[Data_Request['Tỷ lệ Giá vốn chênh lệch']<=0.0000]['Đơn giá nhập'],0)
Data_Request['Giá mặc cả VND']=np.around(Data_Request['Đơn giá nhập']*(1-RateDeal)*1/(1+CostTrans),2)
Data_Request.sort_values(by='Giá mặc cả VND', ascending=False)

#GIÁ MẶC CẢ USD

Data_Request['Giá mặc cả USD']=np.around(Data_Request['Giá mặc cả VND']/RateVNDUSD,2)
Data_Request['Giá vốn mới']=np.around(Data_Request['Giá mặc cả USD']*24000,0)
Data_Request.head()


,ID,ID Item,Description,PCS/PACK,Min qty,Qty request,Unit price,Nhà sản xuất,Giá vốn,Tổng xuất,...,Giá bán giảm,Ngày nhập,Nhà cung cấp,Tổng nhập,Đơn giá nhập,Unit Price VND,Tỷ lệ Giá vốn chênh lệch,Giá mặc cả VND,Giá mặc cả USD,Giá vốn mới
0,04152-YZZA4,"MVP ELEMENT KIT, OIL",NaN,240,NaN,NaN,9.99,TASTI,81221.0,12.0,...,155000.0,2022-08-13,NCC-KHANGTHANG,100,90000.0,270929.0,2.34,75663.72,3.15,75600.0
1,04371-0K110,"SPIDER KIT, UNIVERSA",NaN,564,NaN,NaN,29.22,TASTI,484082.0,115.0,...,550000.0,2022-02-26,NK-TS,100,420282.0,792446.0,0.64,353334.42,14.72,353280.0
2,04465-28520,"PAD KIT, DISC BRAKE",NaN,120,NaN,NaN,93.86,TASTI,1162694.0,1.0,...,1600000.0,2021-10-28,NCC-KHANGTHANG,1,1080000.0,2545483.0,1.19,907964.60,37.83,907920.0
3,04465-YZZQ7,BRAKE PAD,NaN,480,NaN,NaN,39.99,TASTI,1049198.0,629.0,...,1050000.0,2022-09-14,HTAUTOHN,200,1050000.0,1084529.0,0.03,882743.36,36.78,882720.0
4,04465-YZZR5-82,BRAKE PAD,NaN,NaN,NaN,NaN,42.81,FOB SEPTEMBER,1173167.0,12.0,...,1150000.0,2022-06-27,CHPT-MINHANH,8,1130000.0,1161007.0,-0.01,950000.00,39.58,949920.0


In [10]:
# # FILTER: UNIT PRICE by Factories
# Data_Request['Unit price']=np.around(Data_Request['Unit price'],2)
# Data_Request = Data_Request[Data_Request['Ngày nhập']==Data_Request['Ngày nhập_max']].drop('Ngày nhập_max', axis=1)

In [11]:
#SHOW KET QUA
Data_Request_Show=Data_Request.iloc[:,[0,1,2,3,4,5,7,6,17,8,18,10,12,13,14,16,13,15,9,11,19,20,21]].head()
#ID,Unit Price, Unit VND, Giá vốn, %chênh lệch VND, Giá bán, giá giảm, Ngày nhập, Nhà cung cấp, Đơn giá nhập, tổng nhập, tổng xuất, tổng tồn, Giá mặc cả VND, Giá mặc cả USD
Data_Request_Show=Data_Request.sort_values(by='Tổng nhập', ascending=False).reset_index(drop=True)
# Data_Request.loc[Data_Request['ID']=="T24VG4WS"]


In [12]:
# EXPORT FILE DATA
with pd.ExcelWriter('Report.xlsx') as writer:
    Data_Request.to_excel(writer, sheet_name='Data')
    Data_Request_Show.to_excel(writer,sheet_name='Show')
